# Preparación de datos – Customer Value Prediction

## 1) Objetivo
Preparar y transformar los datos del Customer Shopping Dataset para construir
un dataset a nivel cliente que permita entrenar un modelo de Machine Learning
orientado a identificar clientes de alto valor.

El foco de este notebook es:
- Limpieza de datos
- Feature engineering
- Agregación a nivel cliente
- Definición de la variable objetivo

## 2) Importación de librerías

In [15]:
import pandas as pd
import numpy as np

## 3) Carga de datos

Se carga el dataset de transacciones de compras en retail.

In [16]:
df = pd.read_csv('../data/raw/customer_shopping_data.csv')
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


## 4) Exploración inicial del dataset

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [19]:
df.describe()

,age,quantity,price
count,99457.000000,99457.000000,99457.000000
mean,43.427089,3.003429,689.256321
std,14.990054,1.413025,941.184567
min,18.000000,1.000000,5.230000
25%,30.000000,2.000000,45.450000
50%,43.000000,3.000000,203.300000
75%,56.000000,4.000000,1200.320000
max,69.000000,5.000000,5250.000000


## 5) Limpieza de datos

Se eliminan registros incompletos y se validan variables clave
para garantizar la calidad del dataset.

In [20]:
# Verificar valores nulos
df.isnull().sum()

invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

In [22]:
# Eliminar duplicados
df = df.drop_duplicates()
df.shape

(99457, 10)

## 6) Creación de variables adicionales

Se calcula el monto total por transacción.


In [24]:
df['total_amount'] = df['quantity'] * df['price']
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,total_amount
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon,7502.00
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul,5401.53
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity,300.08
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM,15004.25
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon,242.40


## 7) Conversión de la fecha

Se convierte la columna de fecha a formato datetime.


In [28]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='mixed')
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,total_amount
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,2022-05-08,Kanyon,7502.00
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,2021-12-12,Forum Istanbul,5401.53
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,2021-09-11,Metrocity,300.08
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,2021-05-16,Metropol AVM,15004.25
4,I337046,C189076,Female,53,Books,4,60.60,Cash,2021-10-24,Kanyon,242.40


## 8) Agregación a nivel cliente

El modelo se entrenará a nivel cliente, por lo que las transacciones
individuales se agrupan por cliente.

In [37]:
clientes = df.groupby('customer_id').agg(
    Edad=('age', 'mean'),
    Cantidad_Total=('quantity', 'sum'),
    Gasto_Total=('total_amount', 'sum')
).reset_index()

clientes.head()

,customer_id,Edad,Cantidad_Total,Gasto_Total
0,C100004,61.0,5,7502.00
1,C100005,34.0,2,2400.68
2,C100006,44.0,3,322.56
3,C100012,25.0,5,130.75
4,C100019,21.0,1,35.84


## 9) Variables categóricas del cliente

Se agrega información demográfica y de preferencia de pago.


In [38]:
cliente_info = df.groupby('customer_id').agg(
    Genero=('gender', 'first'),
    Metodo_Pago=('payment_method', 'first'),
    Mall_Frecuente=('shopping_mall', 'first')
).reset_index()

clientes = clientes.merge(cliente_info, on='customer_id', how='left')
clientes.head()

,customer_id,Edad,Cantidad_Total,Gasto_Total,Genero,Metodo_Pago,Mall_Frecuente
0,C100004,61.0,5,7502.00,Male,Credit Card,Metrocity
1,C100005,34.0,2,2400.68,Male,Cash,Kanyon
2,C100006,44.0,3,322.56,Male,Credit Card,Cevahir AVM
3,C100012,25.0,5,130.75,Male,Cash,Kanyon
4,C100019,21.0,1,35.84,Female,Credit Card,Metrocity


## 10) Feature engineering

Se crean variables que representan mejor el comportamiento de compra.

In [39]:
clientes['Ticket_Promedio'] = clientes['Gasto_Total']  # una compra por cliente

## 11) Definición de cliente de alto valor

Se define como cliente de alto valor a aquellos cuyo gasto total
se encuentra en el percentil 70 o superior.

In [40]:
percentil_70 = clientes['Gasto_Total'].quantile(0.70)

clientes['cliente_alto_valor'] = np.where(
    clientes['Gasto_Total'] >= percentil_70, 1, 0
)

clientes['cliente_alto_valor'].value_counts()

cliente_alto_valor
0    68705
1    30752
Name: count, dtype: int64

## 12) Dataset final para Machine Learning

Se seleccionan las variables que serán utilizadas como entrada del modelo.

In [42]:
dataset_ml = clientes[[
    'Edad',
    'Cantidad_Total',
    'Ticket_Promedio',
    'Genero',
    'Metodo_Pago',
    'cliente_alto_valor'
]]

dataset_ml.head()

,Edad,Cantidad_Total,Ticket_Promedio,Genero,Metodo_Pago,cliente_alto_valor
0,61.0,5,7502.00,Male,Credit Card,1
1,34.0,2,2400.68,Male,Cash,0
2,44.0,3,322.56,Male,Credit Card,0
3,25.0,5,130.75,Male,Cash,0
4,21.0,1,35.84,Female,Credit Card,0


## 13) Exportación del dataset procesado

In [43]:
dataset_ml.to_csv('../data/processed/clientes_features.csv', index=False)